In [12]:
import matplotlib.pyplot as plt
from skimage import io
from skimage import color
from skimage.transform import resize
import math
from skimage.feature import hog
import numpy as np
import glob
import os
#importing required libraries
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt

import cv2  # importing cv
import imutils

import load_read_name_extractor as lrne
  
  


In [13]:

def fix_mask(img, n_r, n_c, ign):
    for c in range(n_c):
        for r in range(n_r):
            if (img[r][c][3] < 255):
                img[r][c][3] = 0
    return img
    
def top_pix(img, n_r, n_c, ign):
    top = 0
    for r in range(n_r):
        if not(0 in img[r,ign:n_c-ign,3]):
            return r
    return top
def buttom_pix(img, n_r, n_c, ign):
    top = 0
    for r in range(n_r-1,0,-1):
        if not(0 in img[r,ign:n_c-ign,3]):
            return r
    return top
def left_pix(img, n_r, n_c, ignT, ignB):
    top = 0
    for c in range(n_c):
        if not(0 in img[ignT:ignB,c,3]):
            return c
    return top
def right_pix(img, n_r, n_c, ignT, ignB):
    top = 0
    for c in range(n_c-1, 0, -1):
        if not(0 in img[ignT:ignB,c,3]):
            return c
    return top
def instanciate(img, ign):
    n_c = len(img[0])
    n_r = len(img)
    img = fix_mask(img, n_r, n_c, ign)
    top = top_pix(img, n_r, n_c, ign)
    buttom = buttom_pix(img, n_r, n_c, ign)
    right = right_pix(img, n_r, n_c, top + ign, buttom-ign)
    left = left_pix(img, n_r, n_c, top + ign, buttom-ign)
    return top, buttom, left, right

In [14]:

def find_left_corner (img):
    n_c = len(img[0])
    n_r = len(img)
    for c in range(n_c):
        for r in range(n_r):
            if (img[r][c][3] == 255):
                #print("r =" ,r, "c =" ,c)
                return r, c

def find_right_corner (img):
    n_c = len(img[0])
    n_r = len(img)
    for r in range(n_r):
        for c in range(n_c):
            if (img[r][c][3] == 255):
                #print("r =" ,r, "c =" ,c)
                return r, c
def it_rotate_img(img_list, names):
    angle = 0
    for i in range(len(img_list)):
        r1, c1 = find_left_corner (img_list[i])
        r2, c2 = find_right_corner (img_list[i])
        a = r1 - r2
        c = c2 - c1
        try:
            angle = np.degrees(np.arctan(a/c))
        except:
            print(names[i])
        #Decides wether the picture i tilted to the right or 
        # to the left and adds 90 degrees if it is tilted to the left
        #This statement doesnt work for 170707_Bj so we removed this for that flight
        if (c2 < 200):
            img_list[i] = imutils.rotate(img_list[i], angle=-angle-90)
        else:
            img_list[i] = imutils.rotate(img_list[i], angle=-angle)
    return img_list

def it_crop_img(img_list):
    for i in range (len(img_list)):
        current_img = img_list[i]
        top, buttom, left, right = instanciate(current_img, 20)
        print(top, buttom, left, right)
        img_list[i] = current_img[top:buttom, left:right]
        if (0 in img_list[i][:,:, 3]):
            #print ("CROP FAILURE on ", 166+i, "RETRIES WITH IGNORE VALUE 0")
            current_img = img_list[i]
            top, buttom, left, right = instanciate(current_img, 0)
            img_list[i] = current_img[top:buttom, left:right]
            if (0 in img_list[i][:,:, 3]):
                print ("CROP FAILURE on ", 166+i)
    return img_list

def it_hog(img_list):
    for i in range (len(img_list)):
        fd, hog_image = hog(img_list[i], orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, multichannel=True)
        #plt.axis("off")
        #plt.imshow(hog_image, cmap="gray")
        img_list[i] = hog_image
    return img_list

def save_img_list(img_list, path, names):
    for i in range (len(img_list)):
        try:
            plt.imsave(path + names[i][0] + "_" + names[i][1] + '.jpeg', np.ascontiguousarray(img_list[i]))
        except Exception as e:
                print('Save Failure On: ' + names[i][0] + "_" + names[i][1] + '\n' + 'With Failure: ' + str(e))

def reshape(img_list, names):
    r,c = find_mean_shape(img_list)
    for i in range(len(img_list)):
        try:
            img_list[i] = cv2.resize(img_list[i], (c, r))
        except:
            print(print(names[i]))
    return img_list

def find_mean_shape(img_list):
    n = len(img_list)
    mean_c = 0
    mean_r = 0
    for i in range(n):
        mean_r += img_list[i].shape[0]
        mean_c += img_list[i].shape[1]
    return int(np.floor((mean_r/n)/8)*8), int(np.floor((mean_c/n)/8)*8)

            
    

In [15]:
img_list, names = lrne.load_name_and_img('160607_Min/*.tif')


In [16]:
names[0]

'160607_Min\\20160607_mm_ebee_rgb_transparent_mosaic_group1_ROI_61.tif'

In [18]:
list_test = lrne.it_name_extract_labels_from_img(names)


In [19]:
print(list_test[0])

['160607_Min', '61']


In [20]:

def it_process(img_list, names):
    img_list = it_rotate_img(img_list, names)
    img_list = it_crop_img(img_list)
    img_list = reshape(img_list, names)
    name_list = lrne.it_name_extract_labels_from_img(names)
    save_img_list(img_list, "cropped/", name_list)

it_process(img_list, names)


7 161 5 556
8 164 3 549
8 163 3 553
9 162 7 558
11 165 4 561
11 165 3 553
10 164 3 554
9 163 3 554
9 160 5 555
8 160 3 553
8 163 3 552
9 165 4 556
8 163 4 557
8 163 3 558
10 164 3 554
